In [1]:
pip install pandas numpy scikit-learn

Active code page: 1252
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

pd.set_option("display.max_columns", None)

In [16]:
DATA_PATH = "../data/raw/"

orders = pd.read_csv(DATA_PATH + "olist_orders_dataset.csv")
reviews = pd.read_csv(DATA_PATH + "olist_order_reviews_dataset.csv")
order_items = pd.read_csv(DATA_PATH + "olist_order_items_dataset.csv")

# Merge orders + reviews
df = orders.merge(reviews, on="order_id", how="inner")

# Target
df["low_rating"] = (df["review_score"] <= 2).astype(int)

print("Dataset size:", df.shape)

Dataset size: (99224, 15)


In [17]:
orders["order_purchase_timestamp"] = pd.to_datetime(orders["order_purchase_timestamp"])
orders["order_delivered_customer_date"] = pd.to_datetime(orders["order_delivered_customer_date"])
orders["order_estimated_delivery_date"] = pd.to_datetime(orders["order_estimated_delivery_date"])

orders["delivery_delay_days"] = (
    orders["order_delivered_customer_date"] -
    orders["order_estimated_delivery_date"]
).dt.days

df = df.merge(
    orders[["order_id", "delivery_delay_days"]],
    on="order_id",
    how="left"
)

# Clip outliers
df["delivery_delay_days"] = df["delivery_delay_days"].clip(-60, 30)

In [18]:
price_df = (
    order_items.groupby("order_id")[["price", "freight_value"]]
    .sum()
    .reset_index()
)

df = df.merge(price_df, on="order_id", how="left")

In [19]:
df["order_purchase_timestamp"] = pd.to_datetime(df["order_purchase_timestamp"])

df = df.sort_values("order_purchase_timestamp")

split_index = int(len(df) * 0.8)

train_df = df.iloc[:split_index].copy()
test_df = df.iloc[split_index:].copy()

print("Train size:", train_df.shape)
print("Test size:", test_df.shape)

Train size: (79379, 18)
Test size: (19845, 18)


In [20]:
# Attach seller_id
order_seller = order_items[["order_id", "seller_id"]].drop_duplicates()

train_df = train_df.merge(order_seller, on="order_id", how="left")
test_df = test_df.merge(order_seller, on="order_id", how="left")

# Compute seller risk from TRAIN ONLY
seller_risk = (
    train_df.groupby("seller_id")["low_rating"]
    .mean()
    .reset_index()
)

seller_risk.columns = ["seller_id", "seller_historical_risk"]

train_df = train_df.merge(seller_risk, on="seller_id", how="left")
test_df = test_df.merge(seller_risk, on="seller_id", how="left")

global_mean = train_df["low_rating"].mean()

train_df["seller_historical_risk"] = train_df["seller_historical_risk"].fillna(global_mean)
test_df["seller_historical_risk"] = test_df["seller_historical_risk"].fillna(global_mean)

In [21]:
feature_cols = [
    "delivery_delay_days",
    "price",
    "freight_value",
    "seller_historical_risk"
]

X_train = train_df[feature_cols]
y_train = train_df["low_rating"]

X_test = test_df[feature_cols]
y_test = test_df["low_rating"]

print("X_train:", X_train.shape)
print("X_test:", X_test.shape)

X_train: (80372, 4)
X_test: (20196, 4)


In [24]:
print(X_train.isna().sum())
print(X_test.isna().sum())

delivery_delay_days       2490
price                      656
freight_value              656
seller_historical_risk       0
dtype: int64
delivery_delay_days       378
price                     103
freight_value             103
seller_historical_risk      0
dtype: int64
